# Projet de programmation

[Télécharger les données et ce document](progproj.zip)

Prénom et nom de chaque participant:

* 
* 
* 

Si plusieurs participants: Décrire en quelques lignes les contributions majeures de chaque participant.

## Description de l'objectif

* Intégrer la représentation du réseau parisien accessible au vélo et les informations de nivellement
* Définir une fonction permettant, étant donnés deux points $A$ et $B$, de calculer le plus court chemin entre ces deux points (critère de distance uniquement)
* Établir une carte de centralité des arêtes en dénombrant, pour chaque arête, le nombre de chemins les plus courts qui la traversent ([edge betweenness](https://link.springer.com/referenceworkentry/10.1007%2F978-1-4419-9863-7_874#:~:text=Definition,an%20edge%20betweenness%20centrality%20value.))
* Définir une fonction permettant, étant donnés deux points $A$ et $B$, de calculer un chemin entre ces deux points avec un maximum de confort
* Établir une carte de centralité des arêtes d'après ce nouveau critère

Si vous trouvez le projet trop facile: Intégrez ces calculs à OSMnx pour en faire une application web interactive.

## Description des données

In [18]:
import pandas as pd
import json

Les données de cartographie ont été obtenues à l'aide de la bibliothèque [OSMnx](https://github.com/gboeing/osmnx) et les données de nivellement à partir des [données de Paris](https://opendata.paris.fr/explore/dataset/plan-de-voirie-repere-de-nivellement/) (format CSV). Ces données ont été traitées pour faciliter leur utilisation dans le cadre de ce projet.

Les données de nivellement sont au format CSV (colonne `ALTITUDE`) et comportent de nombreuses informations sur les points où le dénivellé a été déterminé (numéro de rue, longitude, latitude):

In [29]:
elevation = pd.read_csv('opendata_paris_nivel_lat_long.csv', index_col=0)

In [30]:
elevation.head(3)

,OBJECTID,NUMERO,IMMEUBLE,DESIGNATION,DENOMINATION,COMPLEMENT,ALTITUDE,ANNEE,X,Y,TYP,geo_shape,geo_point_2d,lat,long
0,14347,rn33088,1,Rue du,CARDINAL MERCIER,#,49.488,1985,599438.429,131177.365,ENSERVICE,"{""type"": ""Point"", ""coordinates"": [2.3288668703...","48.8810365523,2.32886687038",48.881037,2.328867
1,14355,rn39097,26,Boulevard de,STRASBOURG,#,35.152,1985,601405.488,130114.709,ENSERVICE,"{""type"": ""Point"", ""coordinates"": [2.3556839617...","48.8714789852,2.35568396171",48.871479,2.355684
2,14364,rn32111,29,Rue de,LISBONNE,Angle rue Mollien,44.063,1983,598304.037,130793.953,ENSERVICE,"{""type"": ""Point"", ""coordinates"": [2.3134011689...","48.8775865507,2.31340116892",48.877587,2.313401


Les données de cartographie ont été obtenues à l'aide de la commande OSMnx suivante, permettant de décrire sous la forme d'un graphe les voies de Paris empruntables par un vélo:

```Python
ox.graph_from_place('Paris, France', network_type='bike', simplify=True)
```

Ces informations sont mises à votre disposition par l'entremise de deux fichiers de données:

* Un fichier contenant les quelques 20&nbsp;000 nœuds du graphe (`nodes.csv`) et leurs coordonnées géographiques (latitudes et longitudes)
* Un fichier contenant les listes d'adjacence (`adjacency.json`)

In [24]:
nodes = pd.read_csv('nodes.csv', index_col=0)
nodes['nodeid'] = nodes['nodeid'].map(str)

In [25]:
nodes.head(3)

,nodeid,latitude,longitude
0,125730,48.863523,2.410732
1,125742,48.851852,2.401373
2,125747,48.854261,2.414664


Chaque nœud correspond à une intersection dans le réseau urbain (par exemple, le croisement entre la rue de la Verrerie et la rue des Archives) et dispose d'un identifiant unique `nodeid`.

Par exemple, le nœud `125730` peut être visualisé sur [Google Maps](https://www.google.com/maps/place/48.863523,2.410732) ou [Wolfram|Alpha](https://www.wolframalpha.com/input/?i=latitude+48.863523+longitude%092.410732) en entrant ses coordonnées GPS. En particulier, on peut recourir à une URL de la forme https://www.google.com/maps/place/48.863523,2.410732.

Les listes d'adjacence sont stockées sous la forme d'un dictionnaire dont les clefs sont les identifiants `nodeid` des nœuds. À chaque clef/nœud, correspond un dictionnaire dont les clefs sont les nœuds voisins et les valeurs indiquent le nom de la voie et sa longueur (qui peut également être recalculée à l'aide des coordonnées GPS et de la formule de Haversine). Ainsi le node <code>125730</code> est-il connecté à quatre nœuds voisins (<code>268675130</code>, <code>4761621113</code>, <code>244455548</code> et <code>1089076050</code>) via la rue Henri-Duvernois, la rue Louis-Lumière, la rue Louis-Lumière et la rue Louis-Ganne, respectivement.

In [19]:
with open('adjacency.json', 'r') as infile:
    adjacency = json.load(infile)

In [22]:
adjacency['125730']

{'268675130': {'name': 'Rue Henri Duvernois', 'length': 8.707},
 '4761621113': {'name': 'Rue Louis Lumière', 'length': 132.79999999999998},
 '244455548': {'name': 'Rue Louis Lumière', 'length': 105.92499999999998},
 '1089076050': {'name': 'Rue Louis Ganne', 'length': 124.29}}

<img src="Screenshot_2020-11-21_at_23.14.33_annotated.png" />